In [2]:
import cirq
from cirq.circuits import Circuit
import numpy as np
from scipy.stats import unitary_group
from cirq.sim import Simulator
sim = Simulator()

In [5]:
class FCGate:

    def __init__(self, control_0, control_1, target, sub_gate):
        
        self.control_0 = control_0
        self.control_1 = control_1
        self.target = target
        self.sub_gate = sub_gate
                
        
    def build_gate(self, qubits):
        
        # Create Fully Controlled Gate
        num_controls = len(self.control_0) + len(self.control_1)
        control_values = [0] * len(self.control_0) + [1] * len(self.control_1) 
        fc_gate = cirq.ControlledGate(self.sub_gate, num_controls, control_values)

        # Create ordered qubit list for cirq.ControlledGate [c0,...,c1,...,target]
        control_0_qubits = [qubits[i] for i in self.control_0]
        control_1_qubits = [qubits[i] for i in self.control_1]
        target_qubit = [qubits[self.target]]
        ordered_qubits = control_0_qubits + control_1_qubits + target_qubit

        
        return fc_gate.on(*ordered_qubits) 

In [6]:
def state1_to_state2_gate(arr1, arr2, sub_gate):
    N = len(arr1)
    
    control_0 = []
    control_1 = []
    target = None
    for i in range(N):
        
        # if ith qubits have same state -> control qubits
        if arr1[i] == arr2[i]:
            if arr1[i] == 0:
                control_0.append(i)
            else:
                control_1.append(i)
        else:
            target = i
        
    return FCGate(contol_0, control_1, target, sub_gate)

# Example

### Before

In [7]:
qs = [cirq.GridQubit(0,1), cirq.GridQubit(1,1), cirq.GridQubit(2,1), cirq.GridQubit(2,2)] 

circuit = cirq.Circuit([cirq.X.on_each(qs[2]), cirq.I.on_each(qs[1], qs[0], qs[3])])
res = sim.simulate(circuit)
res.dirac_notation()

'|0010⟩'

### After

In [8]:
# Instantiate an FCgate
control_0 = [0, 3]
control_1 = [2]
target = 1 
sub_gate = cirq.X
fc_gate = FCGate(control_0, control_1, target, sub_gate)

circuit = cirq.Circuit([ 
    cirq.X.on(qs[2]), 
    fc_gate.build_gate(qs)
])
res = sim.simulate(circuit)
res.dirac_notation()

'|0110⟩'